A)

Import libraries and load data (table was previously transferred to .csv):

In [1]:
import pandas as pd
import numpy as np
import math

dt = pd.read_csv('data.csv', sep=';').drop('REC',axis=1)
dt.head()

,Age,Income,Student,Credit_rating,Buys_computer
0,<=30,High,No,Fair,No
1,<=30,High,No,Excellent,No
2,31...40,High,No,Fair,Yes
3,>40,Medium,No,Fair,Yes
4,>40,Low,Yes,Fair,Yes


Create function to calculate entropy:

In [2]:
# source: https://www.programcreek.com/python/?CodeExample=calculate+entropy

def calculate_entropy(y):
    """ Calculate the entropy of label array y """
    log2 = lambda x: math.log(x) / math.log(2)
    unique_labels = np.unique(y)
    entropy = 0
    for label in unique_labels:
        count = len(y[y == label])
        p = count / len(y)
        entropy += -p * log2(p)
    return entropy 

Calculation of the total entropy of the data set:

In [3]:
total_entropy = calculate_entropy(dt['Buys_computer'])
total_entropy

0.9709505944546686

Create table, how often a computer is purchased, depending on 'Age':

In [4]:
freq_age_buys_computer = pd.crosstab(index=dt['Age'], columns=dt['Buys_computer'])
freq_age_buys_computer["sum"] = freq_age_buys_computer.sum(axis=1)
freq_age_buys_computer

Buys_computer,No,Yes,sum
Age,,,
31...40,0,6,6
<=30,6,2,8
>40,2,4,6


Create table, how often a computer is purchased, depending on 'Income':

In [5]:
freq_income_buys_computer = pd.crosstab(index=dt['Income'], columns=dt['Buys_computer'])
freq_income_buys_computer["sum"] = freq_income_buys_computer.sum(axis=1)
freq_income_buys_computer

Buys_computer,No,Yes,sum
Income,,,
High,2,3,5
Low,3,4,7
Medium,3,5,8


Create table, how often a computer is purchased, depending on 'Student':

In [6]:
freq_student_buys_computer = pd.crosstab(index=dt['Student'], columns=dt['Buys_computer'])
freq_student_buys_computer["sum"] = freq_student_buys_computer.sum(axis=1)
freq_student_buys_computer

Buys_computer,No,Yes,sum
Student,,,
No,7,4,11
Yes,1,8,9


Create table, how often a computer is purchased, depending on 'Credit_rating':

In [7]:
freq_credit_buys_computer = pd.crosstab(index=dt['Credit_rating'], columns=dt['Buys_computer'])
freq_credit_buys_computer["sum"] = freq_credit_buys_computer.sum(axis=1)
freq_credit_buys_computer

Buys_computer,No,Yes,sum
Credit_rating,,,
Excellent,5,5,10
Fair,3,7,10


Calculation of individual entropy and information gain in Excel:

![title](entropy_gain.png)

Creating functions for a decision tree based on information gain. I also tried to do this myself, but I ran into some problems, so I used a tutorial (see source):

In [8]:
# source: https://medium.com/geekculture/step-by-step-decision-tree-id3-algorithm-from-scratch-in-python-no-fancy-library-4822bbfdd88f

def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset
    total_entr = 0
    
    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset
    
    return total_entr

def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting

def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of feature_value = count of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #should extend the node, so the branch is marked with ?
            
    return tree, train_data

def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset
                
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data_m, label, class_list) #start calling recursion
    return tree

Calculate the information gain of the first step to verify the calculations shown above in Excel:

In [9]:
print("Information gain of 'Age': ", calc_info_gain('Age', dt, 'Buys_computer', dt['Buys_computer'].unique()))
print("Information gain of 'Income': ", calc_info_gain('Income', dt, 'Buys_computer', dt['Buys_computer'].unique()))
print("Information gain of 'Student': ", calc_info_gain('Student', dt, 'Buys_computer', dt['Buys_computer'].unique()))
print("Information gain of 'Credit_rating': ", calc_info_gain('Credit_rating', dt, 'Buys_computer', dt['Buys_computer'].unique()))

Information gain of 'Age':  0.3709505944546686
Information gain of 'Income':  0.0016094970590274649
Information gain of 'Student':  0.22437117627527592
Information gain of 'Credit_rating':  0.03030514483932223


Creating a decision tree:

In [10]:
tree = id3(dt, 'Buys_computer')
print(tree)

{'Age': {'<=30': {'Student': {'No': 'No', 'Yes': 'Yes'}}, '31...40': 'Yes', '>40': {'Credit_rating': {'Fair': 'Yes', 'Excellent': {'Income': {'Low': 'No', 'Medium': {'Student': {'No': 'No', 'Yes': 'Yes'}}}}}}}}


B)

Creating of a decision tree with sklearn (incl. encoding of the data set via OrdinalEncoder):

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder

X = dt.drop('Buys_computer',axis=1)
y = dt.drop(['Age', 'Income', 'Student', 'Credit_rating'], axis=1)

cat_age = ['<=30','31...40','>40']
cat_income = ['Low','Medium','High']
cat_student = ['No','Yes']
cat_credit = ['Fair','Excellent']

enc = OrdinalEncoder(categories=[cat_age, cat_income, cat_student, cat_credit])
X = enc.fit_transform(X)
X = pd.DataFrame(X)
X = X.rename(columns={0: "Age", 1: "Income", 2: "Student", 3: "Credit_rating"})

tree_clf = DecisionTreeClassifier(max_depth=3)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=3)

Export the decision tree to be able to display it afterwards with WebGraphviz:

In [12]:
from sklearn.tree import export_graphviz
 
export_graphviz(
         tree_clf,
         out_file="visual.dot",
         feature_names=['Age', 'Income', 'Student', 'Credit_rating'],
         rounded=True,
         filled=True
 )

![title](sklearn_dtree.png)

The decision tree is mainly different because only binary decisions are made in sklearn. In both cases, 'Age' is taken for the first division (logical, since it is the attribute with the highest information gain). However, in the manually created decision tree, the attribute is split into all 3 branches, and in the decision tree created by sklearn, it is split into 2 branches in the first step.